# Trabalho Prático 3
**Grupo 22**

Alexis Correia - A102495 <br>
João Fonseca - A102512 <br>


## Problema 3

Considere de novo o 1º problema do trabalho TP2  relativo à descrição da cifra $\,\mathsf{A5/1}$ e o FOTS usando BitVec’s que aì foi definido para a componente do gerador de chaves. Ignore a componente de geração final da chave e restrinja o modelo aos três LFSR’s. 
Sejam $\,\mathsf{X}_0, \mathsf{X}_1, \mathsf{X}_2\,$ as variáveis que determinam os estados dos três LFSR’s que ocorrem neste modelo. Como condição inicial  e condição de erro use os predicados

$$\,\mathsf{I} \;\equiv\; (\mathsf{X}_0 > 0)\,\land\,(\mathsf{X}_1 > 0)\,\land\,(\mathsf{X}_2 > 0)\quad$$ 
$$\quad \mathsf{E}\;\equiv\;\neg\,\mathsf{I}$$
                
1. Codifique em “z3”  o SFOTS assim definido.
2. Use o algoritmo PDR “property directed reachability” (codifique-o ou use uma versão pré-existente) e, com ele, tente provar a segurança deste modelo.

## Resolução

Definidos o conjunto dos estados, o estado inicial e a condição de erro, falta definir as transições deste SFOTS. <bl>
Neste caso, as transições são dadas pela definição da cifra **A5/1**.

In [41]:
from pysmt.shortcuts import *
from pysmt.typing import BVType

n0, n1, n2 = 19, 22, 23
cB0, cB1, cB2 = 8, 10, 10
s0 = BV("1110010000000000000", n0)
s1 = BV("1100000000000000000000", n1)
s2 = BV("11100000000000010000000", n2)

In [62]:
def declare(i):
    state = {}
    state['x0'] = Symbol('x0'+str(i), BVType(n0))
    state['x1'] = Symbol('x1'+str(i), BVType(n1))
    state['x2'] = Symbol('x2'+str(i), BVType(n2))
    return state

def init(state):
    A = BVSGT(state['x0'], BV(0,n0))
    B = BVSGT(state['x1'], BV(0,n1))
    C = BVSGT(state['x2'], BV(0,n2))
    return And(A,B,C)

def error(state):
    return Not(init(state))

In [60]:
def tapping(s, x, n):
    r = 0
    for i in range(n):
        if BVExtract(s,i,i)==1:
            r = Xor(r, BVExtract(x,i,i))
    return r

def trans(curr, prox):
    c0 = BVExtract(curr['x0'],cB0, cB0)
    c1 = BVExtract(curr['x1'],cB1, cB1)
    c2 = BVExtract(curr['x2'],cB2, cB2)

    t01 = And(Equals(c0,c1), Not(Equals(c0,c2)),
              Equals(prox['x0'], BVXor(BVLShl(curr['x0'],1),BV(tapping(s0,curr['x0'],n0), n0))),
              Equals(prox['x1'], BVXor(BVLShl(curr['x1'],1),BV(tapping(s1,curr['x1'],n1), n1))),
              Equals(prox['x2'], curr['x2']))         
    t12 = And(Equals(c2,c1), Not(Equals(c0,c1)),
              Equals(prox['x1'], BVXor(BVLShl(curr['x1'],1),BV(tapping(s1,curr['x1'],n1), n1))),
              Equals(prox['x2'], BVXor(BVLShl(curr['x2'],1),BV(tapping(s2,curr['x2'],n2), n2))),
              Equals(prox['x0'], curr['x0'])) 
    t02 = And(Equals(c0,c2), Not(Equals(c0,c1)),
              Equals(prox['x0'], BVXor(BVLShl(curr['x0'],1),BV(tapping(s0,curr['x0'],n0), n0))),
              Equals(prox['x2'], BVXor(BVLShl(curr['x2'],1),BV(tapping(s2,curr['x2'],n2), n2))),
              Equals(prox['x1'], curr['x1']))           
    t012 = And(Equals(c0,c1), Equals(c0,c2),
              Equals(prox['x0'], BVXor(BVLShl(curr['x0'],1),BV(tapping(s0,curr['x0'],n0), n0))),
              Equals(prox['x1'], BVXor(BVLShl(curr['x1'],1),BV(tapping(s1,curr['x1'],n1), n1))),
              Equals(prox['x2'], BVXor(BVLShl(curr['x2'],1),BV(tapping(s2,curr['x2'],n2), n2)))) 
             
    return Or(t01, t02, t12, t012)

In [63]:
def genTrace(init,trans,error,n): #
    with Solver(name="z3") as s:
        X = [declare(i) for i in range(n+1)] 
        I = init(X[0])
        Tks = [trans(X[i],X[i+1]) for i in range(n)]
        E = [error(X[i]) for i in range(n)]
        if s.solve([I,And(Tks),Not(And(E))]):
            for i in range(n):
                print("Estado:",i)
                for v in X[i]:
                    print("          ",v,'=',s.get_value(X[i][v]))
        else:
            print("ERRO")

genTrace(init, trans, error, 64)

Estado: 0
           x0 = 16_19
           x1 = 1854_22
           x2 = 972_23
Estado: 1
           x0 = 32_19
           x1 = 1854_22
           x2 = 1944_23
Estado: 2
           x0 = 32_19
           x1 = 3708_22
           x2 = 3888_23
Estado: 3
           x0 = 32_19
           x1 = 7416_22
           x2 = 7776_23
Estado: 4
           x0 = 32_19
           x1 = 14832_22
           x2 = 15552_23
Estado: 5
           x0 = 64_19
           x1 = 29664_22
           x2 = 15552_23
Estado: 6
           x0 = 128_19
           x1 = 59328_22
           x2 = 15552_23
Estado: 7
           x0 = 128_19
           x1 = 118656_22
           x2 = 31104_23
Estado: 8
           x0 = 256_19
           x1 = 118656_22
           x2 = 62208_23
Estado: 9
           x0 = 512_19
           x1 = 237312_22
           x2 = 62208_23
Estado: 10
           x0 = 1024_19
           x1 = 237312_22
           x2 = 124416_23
Estado: 11
           x0 = 1024_19
           x1 = 474624_22
           x2 = 248832_23
Estado: 

Agora, utilizaremos o PDR, retirado da documentação do ```pysmt```, para provar a segurança deste modelo.

In [64]:
### Documentação
class TransitionSystem(object):
    """Trivial representation of a Transition System."""

    def __init__(self, variables, init, trans):
        self.variables = variables
        self.init = init
        self.trans = trans

def next_var(v):
    """Returns the 'next' of the given variable"""
    return Symbol("next(%s)" % v.symbol_name(), v.symbol_type())

class PDR(object):
    def __init__(self, system):
        self.system = system
        self.frames = [system.init]
        self.solver = Solver()
        self.prime_map = dict([(v, next_var(v)) for v in self.system.variables])

    def check_property(self, prop):
        """Property Directed Reachability approach without optimizations."""
        print("Checking property %s..." % prop)

        while True:
            cube = self.get_bad_state(prop)
            if cube is not None:
                # Blocking phase of a bad state
                if self.recursive_block(cube):
                    print("--> Bug found at step %d" % (len(self.frames)))
                    break
                else:
                    print("   [PDR] Cube blocked '%s'" % str(cube))
            else:
                # Checking if the last two frames are equivalent i.e., are inductive
                if self.inductive():
                    print("--> The system is safe!")
                    break
                else:
                    print("   [PDR] Adding frame %d..." % (len(self.frames)))
                    self.frames.append(TRUE())

    def get_bad_state(self, prop):
        """Extracts a reachable state that intersects the negation
        of the property and the last current frame"""
        return self.solve(And(self.frames[-1], Not(prop)))

    def solve(self, formula):
        """Provides a satisfiable assignment to the state variables that are consistent with the input formula"""
        if self.solver.solve([formula]):
            return And([EqualsOrIff(v, self.solver.get_value(v)) for v in self.system.variables])
        return None

    def recursive_block(self, cube):
        """Blocks the cube at each frame, if possible.

        Returns True if the cube cannot be blocked.
        """
        for i in range(len(self.frames)-1, 0, -1):
            cubeprime = cube.substitute(dict([(v, next_var(v)) for v in self.system.variables]))
            cubepre = self.solve(And(self.frames[i-1], self.system.trans, Not(cube), cubeprime))
            if cubepre is None:
                for j in range(1, i+1):
                    self.frames[j] = And(self.frames[j], Not(cube))
                return False
            cube = cubepre
        return True

    def inductive(self):
        """Checks if last two frames are equivalent """
        if len(self.frames) > 1 and \
           self.solve(Not(EqualsOrIff(self.frames[-1], self.frames[-2]))) is None:
            return True
        return False

    def __del__(self):
        self.solver.exit()

### (TransitionSystem(variables, init, trans), [true_prop, false_prop]) -> true_prop == Not(error)

Para utilizar esta função (pré-definida), precisaremos realizar algumas alterações. Nomeadamente, precisaremos alterar as funções ```init``` e ```trans``` da seguinte forma:

In [67]:
def cBitPDR(x0,x1,x2):
    c0 = BVExtract(x0, cB0, cB0)
    c1 = BVExtract(x1, cB1, cB1)
    c2 = BVExtract(x2, cB2, cB2)
    if ((c0 & c1) | (c1 & c2) | (c0 & c2)):
        r = BV(1,1)
    else:
        r = BV(0,1)
    return r

def tappingPDR(s, x, n):
    r = 0
    for i in range(n):
        if BVExtract(s,i,i)==1:
            r = Xor(r, BVExtract(x,i,i))
    return r

def transPDR(x0,x1,x2,nx0,nx1,nx2):
    c0 = BVExtract(x0,cB0, cB0)
    c1 = BVExtract(x1,cB1, cB1)
    c2 = BVExtract(x2,cB2, cB2)

    t01 = And(Equals(c0,c1), Not(Equals(c0,c2)),
              Equals(nx0, BVXor(BVLShl(x0,1),BV(tappingPDR(s0,x0,n0), n0))),
              Equals(nx1, BVXor(BVLShl(x1,1),BV(tappingPDR(s1,x1,n1), n1))),
              Equals(nx2, x2))         
    t12 = And(Equals(c2,c1), Not(Equals(c0,c1)),
              Equals(nx1, BVXor(BVLShl(x1,1),BV(tappingPDR(s1,x1,n1), n1))),
              Equals(nx2, BVXor(BVLShl(x2,1),BV(tappingPDR(s2,x2,n2), n2))),
              Equals(nx0, x0)) 
    t02 = And(Equals(c0,c2), Not(Equals(c0,c1)),
              Equals(nx0, BVXor(BVLShl(x0,1),BV(tappingPDR(s0,x0,n0), n0))),
              Equals(nx2, BVXor(BVLShl(x2,1),BV(tappingPDR(s2,x2,n2), n2))),
              Equals(nx1, x1))           
    t012 = And(Equals(c0,c1), Equals(c0,c2),
              Equals(nx0, BVXor(BVLShl(x0,1),BV(tappingPDR(s0,x0,n0), n0))),
              Equals(nx1, BVXor(BVLShl(x1,1),BV(tappingPDR(s1,x1,n1), n1))),
              Equals(nx2, BVXor(BVLShl(x2,1),BV(tappingPDR(s2,x2,n2), n2)))) 
             
    return Or(t01, t02, t12, t012)

In [68]:
x0, x1, x2 = Symbol('x0', BVType(n0)), Symbol('x1', BVType(n1)), Symbol('x2', BVType(n2))
nx0, nx1, nx2 = Symbol('nx0', BVType(n0)), Symbol('nx1', BVType(n1)), Symbol('nx2', BVType(n2))
                #next_var(x0), next_var(x1), next_var(x2)

var = [x0, x1, x2]
Init = And(BVSGT(x0, BV(0,n0)), BVSGT(x1, BV(0,n1)), BVSGT(x2, BV(0,n2)))
Trans = transPDR(x0,x1,x2,nx0,nx1,nx2)

obj = TransitionSystem(var, Init, Trans)
pdr = PDR(obj)

# Not(error) \eq Not(Not(init)) \eq init
prop =  And(BVSGT(x0, BV(0,n0)), BVSGT(x1, BV(0,n1)), BVSGT(x2, BV(0,n2)))
pdr.check_property(prop)

Checking property ((0_19 s< x0) & (0_22 s< x1) & (0_23 s< x2))...
   [PDR] Adding frame 1...
--> Bug found at step 2
